<a href="https://colab.research.google.com/github/KamilShape/building_style_recognition/blob/main/computer_vision_my_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import random
import os
import numpy as np

### Connecting to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Getting Images

In [ ]:
styles = ['greek-architecture', 'gothic-architecture', 'baroque-architecture', 'victorian-architecture', 'modern architecture']

In [ ]:
def parse_url(content):
  soup = BeautifulSoup(content, 'html.parser')
  imgs = soup.find_all('img', {'class': 'grid-image'})
  return [img['src'] for img in imgs]

In [ ]:
def get_page(style, page):
  for page in range(1, page + 1):
    url = f'https://www.freeimages.com/search/{style}/{page}'
    try:
      response = requests.get(url)
      content = response.content
      return content
    except:
      print('Error! Wrong url.')

In [ ]:
def get_image(photo_url, path, file_name, style):
  image_response = requests.get(photo_url)
  image_content = image_response.content
  img = Image.open(BytesIO(image_content))
  # print(img.mode)
  if img.mode == 'RGB':
    os.makedirs(f'{path}/{style}', exist_ok=True)
    img.save(f'{path}/{style}/{file_name}', 'png')

In [81]:
train_path = 'drive/MyDrive/architecture_photos/data/train'

In [84]:
def get_photos(style, page, path):
  content = get_page(style, page)
  images = parse_url(content)
  for image in images:
    if len(image) > 5:
      file_name = image.split('/')[-1]
      get_image(image, train_path, file_name, style)


In [85]:
#DOWNLOADING ALL PHOTOS
for style in styles:
  get_photos(style, 10, train_path)

### Test and train data sets

In [86]:
test_path = 'drive/MyDrive/architecture_photos/data/test'

In [87]:
os.makedirs(f'{test_path}')

In [93]:
def create_test_set(styles, train_path, test_path, set_size):
  for style in styles:
    files_folder = os.listdir(f'{train_path}/{style}')
    for n in range(0, round(len(files_folder)*set_size)+1):
      files_folder = os.listdir(f'{train_path}/{style}')
      photo_number = np.random.randint(len(files_folder))
      os.makedirs(f'{test_path}/{style}', exist_ok=True)
      os.replace(f'{train_path}/{style}/{files_folder[photo_number]}', f'{test_path}/{style}/{files_folder[photo_number]}')

In [94]:
create_test_set(styles, train_path, test_path, 0.15)